In [ ]:
!pip install datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import os
import pandas as pd
import numpy as np
import re
import torch
import random
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
import evaluate

In [ ]:
# Load datasets
train_set1 = load_dataset('ai4privacy/pii-masking-300k', split='train')
val_set = load_dataset('ai4privacy/pii-masking-300k', split='validation')
filepath = os.path.join(os.getcwd(), 'updated_combined_mini_df.csv')
train_df2 = pd.read_csv(filepath)
train_set2 = Dataset.from_pandas(train_df2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

1english_openpii_30k.jsonl:   0%|          | 0.00/103M [00:00<?, ?B/s]

dutch_openpii_28k.jsonl:   0%|          | 0.00/102M [00:00<?, ?B/s]

french_openpii_31k.jsonl:   0%|          | 0.00/114M [00:00<?, ?B/s]

...

german_openpii_8k.jsonl:   0%|          | 0.00/29.2M [00:00<?, ?B/s]

italian_openpiii_8k.jsonl:   0%|          | 0.00/28.3M [00:00<?, ?B/s]

spanish_openpii_8k.jsonl:   0%|          | 0.00/27.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/177677 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/47728 [00:00<?, ? examples/s]

In [ ]:
print(type(train_set2))
print(train_set2)


<class 'datasets.arrow_dataset.Dataset'>
Dataset({
    features: ['Unnamed: 0.1', 'Unnamed: 0', 'unmasked_text', 'masked_text', 'privacy_mask'],
    num_rows: 200
})


In [ ]:
# Define label mapping for PII tags
label_map = {
    'O': 0, 'FIRSTNAME_1': 1, 'LASTNAME_1': 2, 'EMAIL_1': 3, 'PHONENUMBER_1': 4,
    'SSN_1': 5, 'JOBTITLE_1': 6, 'SEX_1': 7, 'BUILDINGNUMBER_1': 8, 'STREET_1': 9,
    'DOB_1': 10, 'USERNAME_1': 11, 'AGE_1': 12, 'PREFIX_1': 13, 'ACCOUNTNUMBER_1': 14
}


In [ ]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
model = AutoModelForTokenClassification.from_pretrained('distilroberta-base', num_labels=len(label_map))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Print column names to verify the correct column names
print("Validation Set Columns:", val_set.column_names)
print("Training Set Columns:", train_set2.column_names)


Validation Set Columns: ['source_text', 'target_text', 'privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set']
Training Set Columns: ['Unnamed: 0.1', 'Unnamed: 0', 'unmasked_text', 'masked_text', 'privacy_mask']


In [ ]:
def tokenize_and_align_labels(example):
    # Choose the correct column name based on the dataset
    if 'unmasked_text' in example:
        text_column = 'unmasked_text'
    elif 'source_text' in example:
        text_column = 'source_text'
    else:
        raise ValueError("Neither 'unmasked_text' nor 'source_text' found in the example.")

    # Tokenize the text using the identified column
    tokenized_inputs = tokenizer(example[text_column], truncation=True, padding='max_length', max_length=512)

    # Initialize labels to 'O' (or any other default) for non-masked tokens
    labels = ['O'] * len(tokenized_inputs['input_ids'])

    # Iterate through `privacy_mask` to align labels
    for mask in example.get('privacy_mask', []):
        if isinstance(mask, dict):
            for label_key, redacted_value in mask.items():
                # Skip non-string values in `privacy_mask`
                if not isinstance(redacted_value, str):
                    continue

                label_id = label_map.get(label_key, 0)  # Using 0 for unknown labels
                start_index = example[text_column].find(redacted_value)
                if start_index != -1:
                    end_index = start_index + len(redacted_value)
                    span_tokens = tokenizer(example[text_column][start_index:end_index], add_special_tokens=False).tokens()

                    for i, token in enumerate(tokenizer.tokenize(example[text_column])):
                        if token in span_tokens:
                            labels[i] = label_id

    # Convert labels to numeric format for the model
    numeric_labels = [label_map.get(label, -100) for label in labels]
    tokenized_inputs['labels'] = numeric_labels
    return tokenized_inputs


In [ ]:
# Tokenize both the training and validation datasets using the updated function
encoded_train_set = train_set2.map(tokenize_and_align_labels, batched=False)
small_val_set = val_set.shuffle(seed=123).select(range(40))
encoded_val_set = small_val_set.map(tokenize_and_align_labels, batched=False)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

In [ ]:
# Convert to PyTorch tensors
input_ids = torch.tensor(encoded_train_set['input_ids'])
attention_mask = torch.tensor(encoded_train_set['attention_mask'])
labels = torch.tensor(encoded_train_set['labels'])



In [ ]:
# Define metric for evaluation
metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=2)
    true_labels = labels != -100
    flattened_predictions = predictions[true_labels]
    flattened_labels = labels[true_labels]
    results = metric.compute(references=flattened_labels, predictions=flattened_predictions)
    return results


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results_distilroberta',
    evaluation_strategy='epoch',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_steps=10,
    eval_steps=10,
    gradient_accumulation_steps=2,
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_train_set,
    eval_dataset=encoded_val_set,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


<ipython-input-17-1237cdb5cc7f>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy
0,0.753200,0.046784,1.000000
2,0.001500,0.001706,1.000000


TrainOutput(global_step=36, training_loss=0.211787983464698, metrics={'train_runtime': 2473.2418, 'train_samples_per_second': 0.243, 'train_steps_per_second': 0.015, 'total_flos': 75273875030016.0, 'train_loss': 0.211787983464698, 'epoch': 2.88})

In [19]:
print(encoded_train_set[0])


{'Unnamed: 0.1': 0, 'Unnamed: 0': 200, 'unmasked_text': "Hello Mr. Juston Keebler, This is to remind you of your set cognitive therapy appointment at unit 29164 of Cleveland Street, University on 12/85. Any wish to reschedule, drop us a mail Christ_Harber@gmail.com or ring (239)-4757904. You're charged $0.00b.", 'masked_text': "Hello [PREFIX_1] [FIRSTNAME_1] [LASTNAME_1], This is to remind you of your set [MEDICAL_1] at unit [BUILDINGNUMBER_1] of [STREET_1], [CITY_1] on [DATE_1]. Any wish to reschedule, drop us a mail [EMAIL_1] or ring [PHONENUMBER_1]. You're charged $[AMOUNT_1].", 'privacy_mask': "{'[FIRSTNAME_1]': 'Juston', '[LASTNAME_1]': 'Keebler', '[MEDICAL_1]': 'cognitive therapy appointment', '[BUILDINGNUMBER_1]': '29164', '[STREET_1]': 'Cleveland Street', '[CITY_1]': 'University', '[DATE_1]': '12/85', '[EMAIL_1]': 'Christ_Harber@gmail.com', '[PHONENUMBER_1]': '(239)-4757904', '[AMOUNT_1]': '0.00b'}", 'input_ids': [0, 31414, 427, 4, 1801, 261, 19677, 26539, 6, 152, 16, 7, 8736, 

In [20]:
from collections import Counter

label_counts = Counter()
for example in encoded_train_set:
    label_counts.update(example['labels'])

print("Label Distribution:", label_counts)


Label Distribution: Counter({0: 102400})


In [21]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')  # Example model

# Example encoded dataset (for demonstration, replicate for your dataset):
encoded_train_set = [{'input_ids': tokenizer("Hello Mr. Juston Keebler", return_tensors='pt')['input_ids'].squeeze().tolist(),
                      'attention_mask': [1] * 6,
                      'labels': [0] * 6} for _ in range(5)]

masking_probability = 0.15  # Set masking probability to 15%

for example in encoded_train_set:
    input_ids = example['input_ids']
    labels = input_ids[:]  # Copy of input_ids for labels

    # Mask tokens with the specified probability
    for i in range(len(input_ids)):
        if random.random() < masking_probability:
            example['input_ids'][i] = tokenizer.mask_token_id  # Replace with mask token
        else:
            labels[i] = -100  # Ignore token for loss calculation

    example['labels'] = labels  # Update labels

# Now, re-run your label distribution code
label_counts = Counter()
for example in encoded_train_set:
    label_counts.update(example['labels'])

print("Label Distribution:", label_counts)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Label Distribution: Counter({-100: 40, 101: 2, 17710: 2, 3917: 2, 2239: 2, 7592: 1, 1012: 1})


In [22]:
trainer.evaluate()


{'eval_loss': 0.0017055831849575043,
 'eval_accuracy': 1.0,
 'eval_runtime': 63.7877,
 'eval_samples_per_second': 0.627,
 'eval_steps_per_second': 0.078,
 'epoch': 2.88}

In [23]:
# Save the model and tokenizer
model.save_pretrained('./trained_model_distilroberta')
tokenizer.save_pretrained('./trained_tokenizer_distilroberta')


('./trained_tokenizer_distilroberta/tokenizer_config.json',
 './trained_tokenizer_distilroberta/special_tokens_map.json',
 './trained_tokenizer_distilroberta/vocab.txt',
 './trained_tokenizer_distilroberta/added_tokens.json',
 './trained_tokenizer_distilroberta/tokenizer.json')

In [24]:
# Load the trained model and tokenizer for prediction
model2 = AutoModelForTokenClassification.from_pretrained('./trained_model_distilroberta')
tokenizer2 = AutoTokenizer.from_pretrained('./trained_tokenizer_distilroberta')


In [25]:
# Define redaction function
def redact_column(dataset, column_name):
    pii_pipeline = pipeline('token-classification', model=model2, tokenizer=tokenizer2, aggregation_strategy='max')
    redacted_texts = []

    for text in dataset[column_name]:
        predictions = pii_pipeline(text)
        redacted_text = text
        for entity in sorted(predictions, key=lambda x: x['start'], reverse=True):
            label = entity['entity_group']
            redacted_text = redacted_text[:entity['start']] + f'[{label}]' + redacted_text[entity['end']:]
        redacted_texts.append(redacted_text)

    return redacted_texts


In [28]:
!pip install transformers
from transformers import pipeline


In [29]:
# Test redaction
df = pd.DataFrame({'source_text': ['My name is Jane Doe and you can catch me at 858-333-2923']})
print(redact_column(df, 'source_text'))


['[LABEL_0]']


In [31]:
for i in range(5):  # Iterate using indices
    example = train_set2[i]  # Get the row as a dictionary-like object
    print("Privacy Mask:", example.get('privacy_mask', 'N/A')) # Use get with a default value to handle missing keys


Privacy Mask: {'[FIRSTNAME_1]': 'Juston', '[LASTNAME_1]': 'Keebler', '[MEDICAL_1]': 'cognitive therapy appointment', '[BUILDINGNUMBER_1]': '29164', '[STREET_1]': 'Cleveland Street', '[CITY_1]': 'University', '[DATE_1]': '12/85', '[EMAIL_1]': 'Christ_Harber@gmail.com', '[PHONENUMBER_1]': '(239)-4757904', '[AMOUNT_1]': '0.00b'}
Privacy Mask: {'[JOBTITLE_1]': 'Principal Interactions Executive', '[ORGANIZATION_1]': 'health camp', '[EMAIL_1]': 'Gussie_Champlin62@gmail.com', '[PHONENUMBER_1]': '(974)-0416019'}
Privacy Mask: {'[LASTNAME_1]': 'Schuppe', '[COUNTY_1]': 'Douglas County', '[PHONENUMBER_1]': '(89)-7737 9842'}
Privacy Mask: {'[ZIPCODE_1]': '91724-4349', '[ORGANIZATION_1]': 'Teacher’s Appreciation Day', '[PHONENUMBER_1]': '+77.586.114-4758'}
Privacy Mask: {'[SSN_1]': '756.6471.8923', '[PHONEIMEI_1]': '85-022597-998145-7', '[PHONENUMBER_1]': '016-3146550', '[FIRSTNAME_1]': 'Darren', '[LASTNAME_1]': 'McKenzie-Walker'}


In [32]:
print(train_set2.column_names)

['Unnamed: 0.1', 'Unnamed: 0', 'unmasked_text', 'masked_text', 'privacy_mask']


In [33]:
print(encoded_train_set[:5])

[{'input_ids': [103, 7592, 2720, 1012, 2074, 2239, 103, 15878, 103, 102], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [101, -100, -100, -100, -100, -100, 17710, -100, 3917, -100]}, {'input_ids': [101, 103, 2720, 1012, 2074, 2239, 17710, 15878, 103, 102], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [-100, 7592, -100, -100, -100, -100, -100, -100, 3917, -100]}, {'input_ids': [101, 7592, 2720, 1012, 2074, 103, 103, 15878, 3917, 102], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, 2239, 17710, -100, -100, -100]}, {'input_ids': [101, 7592, 2720, 1012, 2074, 2239, 17710, 15878, 3917, 102], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100]}, {'input_ids': [103, 7592, 2720, 103, 2074, 103, 17710, 15878, 3917, 102], 'attention_mask': [1, 1, 1, 1, 1, 1], 'labels': [101, -100, -100, 1012, -100, 2239, -100, -100, -100, -100]}]


In [34]:
# Test redaction
df = pd.DataFrame({'source_text': ['My name is Jane Doe']})
print(redact_column(df, 'source_text'))

['[LABEL_0]']
